In [6]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Custom Model Implementations

In this notebook, we'll explore implementing custom models using both the functional and subclassing approaches in TensorFlow/Keras.

## Wide and Deep Model

### Functional Implementation

In [10]:
# Define inputs
input_a = tf.keras.layers.Input(shape=[1], name="Wide_Input")
input_b = tf.keras.layers.Input(shape=[1], name="Deep_Input")

# Define deep path
hidden_1 = tf.keras.layers.Dense(30, activation="relu")(input_b)
hidden_2 = tf.keras.layers.Dense(30, activation="relu")(hidden_1)

# Define merged path
concat = tf.keras.layers.concatenate([input_a, hidden_2])
output = tf.keras.layers.Dense(1, name="Output")(concat)

# Define another output for the deep path
aux_output = tf.keras.layers.Dense(1, name="aux_Output")(hidden_2)

# Build the model
model_func = tf.keras.models.Model(inputs=[input_a, input_b], outputs=[output, aux_output])

model_func.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Deep_Input (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 dense_7 (Dense)             (None, 30)                   60        ['Deep_Input[0][0]']          
                                                                                                  
 Wide_Input (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 dense_8 (Dense)             (None, 30)                   930       ['dense_7[0][0]']             
                                                                                            

### Subclassing Implementation

In [22]:

# Inherit from the Model base class
class WideAndDeepModel(tf.keras.models.Model):
    def __init__(self, units=30, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = tf.keras.layers.Dense(units, activation=activation)
        self.hidden2 = tf.keras.layers.Dense(units, activation=activation)
        self.main_output = tf.keras.layers.Dense(1)
        self.aux_output = tf.keras.layers.Dense(1)

    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = tf.keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)

        return main_output, aux_output

# Create an instance of the model
model_sub = WideAndDeepModel()

# Build the model by passing a sample input
sample_input_A = tf.zeros((1, 1))
sample_input_B = tf.zeros((1, 1))
_ = model_sub([sample_input_A, sample_input_B])

# Now, we can call summary()
model_sub.summary()

Model: "wide_and_deep_model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_29 (Dense)            multiple                  60        
                                                                 
 dense_30 (Dense)            multiple                  930       
                                                                 
 dense_31 (Dense)            multiple                  32        
                                                                 
 dense_32 (Dense)            multiple                  31        
                                                                 
Total params: 1053 (4.11 KB)
Trainable params: 1053 (4.11 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## ResNet Model

This model implements a ResNet architecture, a popular deep learning model known for its effectiveness in image classification tasks. It consists of convolutional layers, batch normalization, identity blocks, and global average pooling.

### IdentityBlock Class

Defines a single identity block within the ResNet architecture. This block contains convolutional layers, batch normalization, and skip connections.

In [7]:
# Define IdentityBlock class
class IdentityBlock(tf.keras.Model):
    def __init__(self, filters, kernel_size):
        super(IdentityBlock, self).__init__(name='')
        self.conv1 = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.act = tf.keras.layers.Activation('relu')
        self.add = tf.keras.layers.Add()

    def call(self, input_tensor):
        x = self.conv1(input_tensor)
        x = self.bn1(x)
        x = self.act(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.add([x, input_tensor])
        x = self.act(x)
        return x


### ResNet Class

The main ResNet model, which incorporates multiple identity blocks. It follows a similar structure to the IdentityBlock class, with additional layers for global average pooling and classification.

In [8]:
# Define ResNet class
class ResNet(tf.keras.Model):
    def __init__(self, num_classes):
        super(ResNet, self).__init__()
        self.conv = tf.keras.layers.Conv2D(64, 7, padding='same')
        self.bn = tf.keras.layers.BatchNormalization()
        self.act = tf.keras.layers.Activation('relu')
        self.max_pool = tf.keras.layers.MaxPool2D((3, 3))
        self.id1a = IdentityBlock(64, 3)
        self.id1b = IdentityBlock(64, 3)
        self.global_pool = tf.keras.layers.GlobalAveragePooling2D()
        self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.conv(inputs)
        x = self.bn(x)
        x = self.act(x)
        x = self.max_pool(x)
        x = self.id1a(x)
        x = self.id1b(x)
        x = self.global_pool(x)
        return self.classifier(x)


### Training

In [9]:
# Utility function to preprocess the dataset
def preprocess(features):
    return tf.cast(features['image'], tf.float32) / 255., features['label']

# Create a ResNet instance with 10 output units for MNIST
resnet = ResNet(10)
resnet.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Load and preprocess the dataset
dataset = tfds.load('mnist', split=tfds.Split.TRAIN, data_dir='./data')
dataset = dataset.map(preprocess).batch(32)

# Train the model
resnet.fit(dataset, epochs=1)


Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to ./data/mnist/3.0.1. Subsequent calls will reuse this data.
1875/1875 [==============================] - 256s 135ms/step - loss: 0.1317 - accuracy: 0.9638


## VGG Model

### VGG block

In [26]:
class Block(tf.keras.Model):
    def __init__(self, filters, kernel_size, repetitions, pool_size=2, strides=2):
        super(Block, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.repetitions = repetitions

        # Define convolutional layers based on the number of repetitions
        for i in range(repetitions):
            # Define a Conv2D layer with specified filters, kernel_size, activation, and padding.
            vars(self)[f'conv2D_{i}'] = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, padding='same', activation='relu')

        # Define the max pooling layer that will be added after the Conv2D blocks
        self.max_pool = tf.keras.layers.MaxPooling2D(pool_size=pool_size, strides=strides)

    def call(self, inputs):
        # Access the first convolutional layer (conv2D_0)
        conv2D_0 = self.conv2D_0

        # Connect the conv2D_0 layer to the input
        x = conv2D_0(inputs)

        # Iterate through the remaining convolutional layers (conv2D_i) from 1 to `repetitions`
        for i in range(1, self.repetitions):
            # Access the current convolutional layer (conv2D_i)
            conv2D_i = vars(self)[f'conv2D_{i}']

            # Connect the current convolutional layer to the previous layer (x)
            x = conv2D_i(x)

        # Apply max pooling after all convolutional layers
        max_pool = self.max_pool(x)

        return max_pool


### VGG Class

In [24]:
class MyVGG(tf.keras.Model):

    def __init__(self, num_classes):
        super(MyVGG, self).__init__()

        # Define blocks of VGG with specific configurations
        self.block_a = Block(filters=64, kernel_size=3, repetitions=2)
        self.block_b = Block(filters=128, kernel_size=3, repetitions=2)
        self.block_c = Block(filters=256, kernel_size=3, repetitions=3)
        self.block_d = Block(filters=512, kernel_size=3, repetitions=3)
        self.block_e = Block(filters=512, kernel_size=3, repetitions=3)

        # Classification head
        # Flatten layer to transform 3D tensor into 1D tensor for classification
        self.flatten = tf.keras.layers.Flatten()
        # Fully connected layer with 256 units and ReLU activation
        self.fc = tf.keras.layers.Dense(256, activation='relu')
        # Softmax classifier for the output layer
        self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        # Connect layers sequentially
        x = self.block_a(inputs)
        x = self.block_b(x)
        x = self.block_c(x)
        x = self.block_d(x)
        x = self.block_e(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = self.classifier(x)
        return x


### Training

In [ ]:
# Download the dataset
dataset = tfds.load('cats_vs_dogs', split=tfds.Split.TRAIN, data_dir='data/')

# Initialize VGG with the number of classes
vgg = MyVGG(num_classes=2)

# Compile with losses and metrics
vgg.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define preprocessing function
def preprocess(features):
    # Resize and normalize
    image = tf.image.resize(features['image'], (224, 224))
    return tf.cast(image, tf.float32) / 255., features['label']

# Apply transformations to dataset
dataset = dataset.map(preprocess).batch(32)

# Train the custom VGG model
vgg.fit(dataset, epochs=10)

  7/727 [..............................] - ETA: 3:19:22 - loss: 0.6852 - accuracy: 0.5536